<a href="https://colab.research.google.com/github/RamosMN/Proyectos/blob/main/Youtube_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esta primera práctica consiste en entender cómo hacer query's a la API de Youtube. El documento está basado en el video de StrataScratch titulado "Working with APIs in Python [For Your Data Science Project]" que se encuentra en el siguiente enlace: https://www.youtube.com/watch?v=fklHBWow8vE

In [19]:
# Importo librerías
import requests
import numpy as np
import pandas as pd
import time
from getpass import getpass # para ocultar key

In [6]:
# Keys
API_KEY = getpass('API KEY:')
Channel_ID = 'UCweCc7bSMX5J4jEH7HFImng' # GMHikaru, un canal de ajedrez popular que me interesa analizar

API KEY:··········


In [7]:
#API CALL
pageToken = ''
url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channelId='+Channel_ID+'&part=snippet,id'+'&order=date&maxResults=100'+pageToken
Respuesta = requests.get(url).json()


In [9]:
# Respuesta   # los datos están en la key 'items'

Ides = []
Fecha = []

for video in Respuesta['items']:
  if video['id']['kind'] == 'youtube#video':

    Id = video['id']['videoId']
    Date = video['snippet']['publishTime']
    Date = Date.split('T')[0]

    Ides.append(Id)
    Fecha.append(Date)

In [24]:
np.shape(Ides), np.shape(Respuesta['items'])

((50,), (50,))

In [31]:
#Segunda API CALL
pageToken = ''

for Ids in Ides:
  url = 'https://www.googleapis.com/youtube/v3/videos?key='+API_KEY+'&id='+Ids+'&maxResults=100'+'&part=snippet, statistics'+pageToken
  Respuesta2 = requests.get(url).json()

In [33]:
view = Respuesta2['items'][0]['statistics']['viewCount']
like = Respuesta2['items'][0]['statistics']['likeCount']
fav = Respuesta2['items'][0]['statistics']['favoriteCount']
comm = Respuesta2['items'][0]['statistics']['commentCount']
print(view, like, fav, comm)

96042 2673 0 233


Por cada video quiero guardar esa info en una tabla, para eso se crea un dataframe en pandas vacío con el nombre de las columnas correspondiente

In [14]:
videos = Respuesta2['items']
views, likes, favs, comms = [], [], [], []
for video in videos:
  views.append( video['statistics']['viewCount'] )
  likes.append( video['statistics']['likeCount'] )
  favs.append( video['statistics']['favoriteCount'] )
  comms.append( video['statistics']['commentCount'] )

In [20]:
np.shape(videos)

(1,)